In [11]:
import sys

if 'sympy' not in sys.modules:
  assert('Install sympy with - pip install sympy')

from sympy import Matrix, Symbol, sqrt, solve, Eq, expand, pi

def crossP(vec1, vec2):
  return vec1.col(0).cross(vec2)

## Angabe des Systems
![alternatvie text](src/system.png)

## Angabe der Kräfte

$$ F_C = \sqrt(2)LC $$
$$ qV_1(s_1) = q $$
$$ qV_3(s_3) = \frac{qx_3}{L} $$
$$ qz_4(s_4) = q $$

In [12]:
# DEFINE ALL VARIABLES
q = Symbol('q')
L = Symbol('L')
Ay = Symbol('A_y')
Cx = Symbol('C_x')
Cy = Symbol('C_y')
Dx = Symbol('D_x')
Ex = Symbol('E_x')
Fx = Symbol('F_x')

In [13]:
# SETUP ALL VECTORS
R_1 = Matrix([sqrt(2)/2 * 3*pi*L*q/4, -sqrt(2)/2 * 3*pi*L*q/4,0])
R_2 = Matrix([0, -sqrt(3)*L/2, 0])
R_3 = Matrix([0,-sqrt(3)*q/4,0])
x_R3 = Matrix([L/sqrt(3),0,0])

F_c = Matrix([q*L, q*L,0])

A = Matrix([0,Ay,0])
C = Matrix([-Cx,-Cy, 0])
D = Matrix([Dx,0,0])
E = Matrix([Ex,0,0])
F = Matrix([Fx,0,0])

----
### Berechnung von Auflager $A_y$
$ +\uparrow \sum{F_y} \stackrel{!}{=} 0 $

In [14]:
tmp = A[1]+R_1[1]+R_2[1]+F_c[1]+R_3[1]
A[1] = expand(solve(tmp, Ay)[0])
A

Matrix([
[                                                    0],
[-L*q + 3*sqrt(2)*pi*L*q/8 + sqrt(3)*L/2 + sqrt(3)*q/4],
[                                                    0]])

-----
### Berechnung von Auflager $D_x$

In [15]:
x_Dx = Matrix([0,L/2,0])
tmp = (crossP(x_R3, R_3) + crossP(x_Dx, D)).row(2)

D[0] = solve(tmp, Dx).popitem()[1]
D

Matrix([
[-q/2],
[   0],
[   0]])

-----
### Berechnung von Auflager $C$
#### 1. $C_y$

In [16]:
t = solve(C[1]+F_c[1]+R_3[1], Cy)
C[1] = -expand(t[0])
C

Matrix([
[              -C_x],
[-L*q + sqrt(3)*q/4],
[                 0]])

#### 2. $C_x$

In [17]:
x_Dx = Matrix([0, L, 0])
x_R3 = Matrix([-L/(2*sqrt(3)),0,0])
d_Fc = Matrix([-sqrt(3)*L/2, L/2, 0])
d_C  = Matrix([-sqrt(3)*L/2, L/2, 0])

tmp = (crossP(x_Dx, D)+crossP(x_R3, R_3)+crossP(d_Fc, F_c)+crossP(d_C, C)).row(2)
C[0] = -solve(tmp, Cx).popitem()[1]

C


Matrix([
[        -L*q + q/2],
[-L*q + sqrt(3)*q/4],
[                 0]])

-----
### Berechnung von Auflager $F_x$
$ \stackrel{+}{\rightarrow} \sum{F_x} \stackrel{!}{=} 0 $

$ C_x + F_x + D_x + F_x = 0 $

In [18]:
# tmp = solve(C[0]+F_c[0]+D[0]+F[0], Fx)
tmp = C[0]+F_c[0]+D[0]+F[0]
F[0] = solve(tmp, Fx)
F

Matrix([
[0],
[0],
[0]])

----
### Berechnung von Auflager $E_x$
$ \stackrel{+}{\rightarrow} \sum{F_x} \stackrel{!}{=} 0 $

In [19]:
tmp = D[0]+F[0]+F_c[0]+R_1[0]+E[0]
E[0] = expand(solve(tmp, Ex)[0])
E

Matrix([
[-3*sqrt(2)*pi*L*q/8 - L*q + q/2],
[                              0],
[                              0]])

----
### Kontrolle durch Summe der Momente um Gelenk $C$
$ \stackrel{\curvearrowleft}{+} \sum{M_C} = 0 $

In [20]:
d_R1 = Matrix([-(3*L/2 - 3*L/2*1/sqrt(2)),-(L+3*L/2*1/sqrt(2)),0])
d_R2 = Matrix([-(L+sqrt(3)*L/4),L/4,0])
d_R3 = Matrix([L/sqrt(3),0,0])

d_A = Matrix([-(L+3*L/2), -3*L/2, 0])
d_D = Matrix([sqrt(3)*L/2, L/2,0])
d_E = Matrix([-(L+sqrt(3)*L/2),L/2,0])
d_F = Matrix([sqrt(3)*L/2, -L/2,0])

tmp = crossP(d_A, A) + crossP(d_D,D) + crossP(d_E, E) + crossP(d_F, F) + crossP(d_R1, R_1) + crossP(d_R2, R_2) + crossP(d_R3, R_3)
expand(tmp[2])

3*sqrt(2)*pi*L**2*q/16 + 3*L**2*q - 3*sqrt(3)*L**2/4 + 3*L**2/8 - 5*sqrt(3)*L*q/8 - L*q/4